In [2]:
import pandas as pd

In [9]:
network_normal_data = pd.read_csv("dataset/Network datatset/csv/normal.csv")

In [11]:
physical_normal_data = pd.read_csv("dataset/Physical dataset/phy_norm.csv", encoding='utf-16')

In [12]:
physical_normal_data # Is going to need a lot of preprocssing ... use `file` on all physical files to verify their encoding... a function would be great I think

,Time\tTank_1\tTank_2\tTank_3\tTank_4\tTank_5\tTank_6\tTank_7\tTank_8\tPump_1\tPump_2\tPump_3\tPump_4\tPump_5\tPump_6\tFlow_sensor_1\tFlow_sensor_2\tFlow_sensor_3\tFlow_sensor_4\tValv_1\tValv_2\tValv_3\tValv_4\tValv_5\tValv_6\tValv_7\tValv_8\tValv_9\tValv_10\tValv_11\tValv_12\tValv_13\tValv_14\tValv_15\tValv_16\tValv_17\tValv_18\tValv_19\tValv_20\tValv_21\tValv_22\tLabel_n\tLabel
0,09/04/2021 11:30:50\t0\t0\t0\t0\t0\t0\t0\t0\tf...
1,09/04/2021 11:30:51\t0\t0\t0\t0\t0\t0\t0\t0\tf...
2,09/04/2021 11:30:52\t0\t0\t0\t0\t0\t0\t0\t0\tf...
3,09/04/2021 11:30:53\t0\t0\t0\t0\t0\t0\t0\t0\tf...
4,09/04/2021 11:30:54\t0\t0\t0\t0\t0\t0\t0\t0\tf...
...,...
3423,09/04/2021 12:27:53\t4\t1037\t7\t452\t300\t432...
3424,09/04/2021 12:27:54\t3\t1059\t7\t439\t296\t431...
3425,09/04/2021 12:27:55\t4\t1091\t8\t429\t291\t431...
3426,09/04/2021 12:27:56\t4\t1131\t8\t417\t288\t421...


In [ ]:
# TODO:
# Make a function to clean the physical datasets
# Make datasets for train and test with samples (I suspect using everything would be too much for my poor cpu)
# Train some models: Isolation Forest, LOF, KNN, SVM, Naive Bayes
# Generate basic statistics and visualisations
# This would be enough for 26.10 and should not be long
# Why not train a MLP :eyes: ?